# Graph Frames Example

<font color='steelblue'>
<h3>
<span style="font-family:Arial; font-size:1.6em;">
Example of using Spark GraphFrames<br>
<br>

</span>
</h3>
</font>

<font color='gray'>
<span style="font-family:Arial; font-size:1.4em;">
    <strong>Will look at the following</strong>:
    <ul>
        <li><strong>Create Graph:</strong> Using vertices and edges dataframes, build a GraphFrame</li>
        <li><strong>Basic Queries:</strong> Run basic queries on the graph frame and the dataframes</li>
        <li><strong>Connected Components:</strong> Run the connected components algorithm on the graphframe</li>
        <li><strong>Motif Finding:</strong> Build and explore complex relations in the graph</li>
        <li><strong>Triangle Counting:</strong> Triangle count for each vertex</li>
        <li><strong>Page Rank:</strong> Run page rank algorithm on the graph frame</li>
    </ul>
</span>
</font>

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("GraphFrames").getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("Error")

In [ ]:
# GraphFrames requires you to set a directory where it can save checkpoints
sc.setCheckpointDir('labGraphFrames')

In [ ]:
from graphframes import *
import pandas as pd
defaultRows = 999
pd.options.display.max_rows = defaultRows

<span style="text-align: center">
<h1>Vertices and Edges for the Graph</h1>
<img align="center" src='../datasets/labRelations.png'>
</span>

In [ ]:
# Create Vertices Dataframe
vertices = spark.createDataFrame([('1', 'Carter', 'Derrick', 50), 
                                  ('2', 'May', 'Derrick', 26),
                                 ('3', 'Mills', 'Jeff', 80),
                                  ('4', 'Hood', 'Robert', 65),
                                  ('5', 'Banks', 'Mike', 93),
                                 ('98', 'Berg', 'Tim', 28),
                                 ('99', 'Page', 'Allan', 16)],
                                 ['id', 'name', 'firstname', 'age'])

In [ ]:
# Create Edges Dataframe
edges = spark.createDataFrame([('1', '2', 'friend'), 
                               ('2', '1', 'friend'),
                              ('3', '1', 'friend'),
                              ('1', '3', 'friend'),
                               ('2', '3', 'follows'),
                               ('3', '4', 'friend'),
                               ('4', '3', 'friend'),
                               ('5', '3', 'friend'),
                               ('3', '5', 'friend'),
                               ('4', '5', 'follows'),
                              ('98', '99', 'friend'),
                              ('99', '98', 'friend')],
                              ['src', 'dst', 'type'])

## Creating GraphFrames

Create GraphFrames from vertex and edge DataFrames.

* Vertex DataFrame: A vertex DataFrame should contain a special column named "id" which specifies unique IDs for each vertex in the graph.
* Edge DataFrame: An edge DataFrame should contain two special columns: "src" (source vertex ID of edge) and "dst" (destination vertex ID of edge).

Both DataFrames can have arbitrary other columns. Those columns can represent vertex and edge attributes.

In [ ]:
g = GraphFrame(vertices, edges)## Take a look at the DataFrames

## Basic GraphFrame and Dataframe queries

In [ ]:
# Display the vertices
g.vertices.toPandas().head(defaultRows)

In [ ]:
# show the edges - note each bidirectional relations are different edges
g.edges.toPandas().head(defaultRows)

In [ ]:
# Degrees count the number of edges at each vertex (includes both incoming and outgoing)
g.degrees.toPandas().head(defaultRows)

In [ ]:
g.inDegrees.toPandas().head(defaultRows)

## Connected Components

* Connected Components algorithm finds insolated clusters in the graph
* These clusters are sets of connected vertices in the graph where each vertex is reachable from other vertex in the same set.


In [ ]:
# here we have 2 unique values
g.connectedComponents().show()

In [ ]:
# filter for age greater than 30
g.vertices.filter("age > 30").show()

In [ ]:
# Find all vertices where the indegree is greater than or equal to 2
g.inDegrees.filter("inDegree >= 2").sort("inDegree", ascending=False).show()

In [ ]:
# Find all edges where label is friend
g.edges.filter('type == "friend"').show()

## Motif finding
<br>
<span style="font-family:times, serif; font-size:14pt; font-style:bold">
<ul>
    <li>Using motifs you can build more complex relationships involving edges and vertices</li>
    <li>The following cell finds the pairs of vertices with edges in both directions between them</li>
    <li>The result is a DataFrame, in which the column names are given by the motif keys</li>
    </ul>
The following example will search for pairs of vertices a,b connected by edge e and pairs of vertices b,c connected by edge e2. <br><br>
</span>

In [ ]:
g.find("(a)-[e]->(b); (b)-[e2]->(c)").toPandas().head(50)

## Mutual Friends
<br>
<span style="font-family:times, serif; font-size:14pt; font-style:bold">
<ul>
    <li>As an example we can try to find the mutual friends for any pair of users a and c</li>
    <li>In order to be a mutual friend b, b must be a friend with both a and c (and not just followed by c, for example)</li>
    </ul>
</span>

In [ ]:
mutualFriends = g.find("(a)-[]->(b); (b)-[]->(c); (c)-[]->(b); (b)-[]->(a)").dropDuplicates()
mutualFriends.toPandas().head(20)

## Triangle Counting
<br>
<span style="font-family:times, serif; font-size:14pt; font-style:bold">
Triangle counting is commonly used as community detection and counting in a social network graph. <br>
A triangle is a set of three vertices, where each vertex has a relationship to the other two vertices in the triangle. <br>
    Returns a dataframe.
</span>

In [ ]:
g.triangleCount().show()

## Page Rank
<br>
<span style="font-family:times, serif; font-size:14pt; font-style:bold">
Algorithm assigns each incoming edge into a vertex with a score.<br>
Each incoming edge into a vertex represents the relavance of that vertex within the graph<br>
If a person is followed by many other persons then that person is ranked highly<br>
Parameters to pageRank:
    <ul>
        <li>maxIter – the number of iterations of page rank to run</li>
        <li>resetProbability - the random reset probability – the lower it is, the bigger the score spread between the winners and losers will be – valid ranges are from 0 to 1</li>
    </ul>
</span>

In [ ]:
pr = g.pageRank(resetProbability=0.15, maxIter = 10) ## look at the pagerank score for every vertex
pr.vertices.show()## look at the weight of every edge

In [ ]:
g.vertices.show()

In [ ]:
g.edges.show()

In [ ]:
pr.edges.show()